<a href="https://colab.research.google.com/github/alonzojp/Magnolia-Beach-Model/blob/main/Functionized_Magnolia_Beach_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Imports
import pandas as pd
import plotly.graph_objects as go
import math

import warnings
warnings.filterwarnings("ignore")

In [2]:
# Combine Port Lavaca & OConnor data together
def generate_ports_df(LAVACA_DATA_PATH, OCONNOR_DATA_PATH):
  lavaca_df = pd.read_csv(LAVACA_DATA_PATH)[:-11]
  oconnor_df = pd.read_csv(OCONNOR_DATA_PATH)[:-11]
  ports_df = pd.concat([lavaca_df, oconnor_df], axis=1)

  ports_df.index = lavaca_df['#date+time']
  ports_df.drop(columns=['#date+time'], inplace=True)
  ports_df.index = pd.to_datetime(ports_df.index, format='%m-%d-%Y %H:%M')

  ports_df[ports_df.columns] = ports_df[ports_df.columns].apply(pd.to_numeric, errors='coerce')

  ports_df = ports_df.drop(columns = ['057-wsd', '057-wdr'])
  ports_df.drop(ports_df['2016-10-30':'2016-11-08'].index, inplace=True) # 033-wsd error

  return ports_df

In [3]:
# Visualization of all data in dataset
def graph_data(df):
  for column in df.columns:
    fig = go.Figure(data=go.Scatter(x=df.index, y=df[column], mode='lines', line_color='#003f5c', name = ""))
    fig.update_traces(connectgaps=False)
    fig.update_layout(title = column, xaxis_title="", yaxis_title="")
    fig.show()

In [4]:
# Converts wsd and wdr to X and Y wind data
def convert_winds(ports_df):
  ports = ['033']
  for port in ports:
    x_wind = []
    y_wind = []
    for i in range(0, len(ports_df)):
      radius = ports_df[port + '-wsd'][i]
      theta = ports_df[port + '-wdr'][i]
      theta = theta * (math.pi / 180)
      x_wind.append(radius * math.cos(theta))
      y_wind.append(radius * math.sin(theta))
    ports_df[port + "-Xwind"] = x_wind
    ports_df[port + "-Ywind"] = y_wind
    del ports_df[port + '-wsd']
    del ports_df[port + '-wdr']
  return ports_df

In [5]:
# Creates estimated Magnolia data
def create_magnolia_data(ports_df):
  ports_df['mag-pwl'] = (ports_df['033-pwl'] + ports_df['057-pwl']) / 2
  ports_df['mag-surge'] = (ports_df['033-surge'] + ports_df['057-surge']) / 2
  ports_df['mag-harmwl'] = (ports_df['033-harmwl'] + ports_df['057-harmwl']) / 2
  return ports_df

In [6]:
LAVACA_DATA_PATH = '/content/Lavaca Data 09_24.csv' #033
OCONNOR_DATA_PATH = '/content/OConnor Data 09_24.csv' #057

ports_df = generate_ports_df(LAVACA_DATA_PATH, OCONNOR_DATA_PATH)
ports_df = create_magnolia_data(ports_df)
ports_df = convert_winds(ports_df)
# graph_data(ports_df)
ports_df

,033-pwl,033-harmwl,033-surge,057-pwl,057-harmwl,057-surge,mag-pwl,mag-surge,mag-harmwl,033-Xwind,033-Ywind
#date+time,,,,,,,,,,,
2010-01-01 00:00:00,0.070,-0.149,0.219,0.153,0.032,0.121,0.1115,0.1700,-0.0585,NaN,NaN
2010-01-01 01:00:00,0.124,-0.083,0.207,0.191,0.073,0.118,0.1575,0.1625,-0.0050,NaN,NaN
2010-01-01 02:00:00,0.182,-0.017,0.199,0.216,0.108,0.108,0.1990,0.1535,0.0455,NaN,NaN
2010-01-01 03:00:00,0.224,0.042,0.182,0.292,0.133,0.159,0.2580,0.1705,0.0875,NaN,NaN
2010-01-01 04:00:00,0.202,0.093,0.109,0.267,0.148,0.119,0.2345,0.1140,0.1205,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
2025-01-01 19:00:00,NaN,-0.231,NaN,NaN,-0.207,NaN,NaN,NaN,-0.2190,NaN,NaN
2025-01-01 20:00:00,NaN,-0.272,NaN,NaN,-0.195,NaN,NaN,NaN,-0.2335,NaN,NaN
2025-01-01 21:00:00,NaN,-0.295,NaN,NaN,-0.169,NaN,NaN,NaN,-0.2320,NaN,NaN
